In [12]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import pandas as pd
import os

In [21]:
if not os.path.isfile('datasets/calendar.csv.gz'):
    !curl -OL http://data.insideairbnb.com/spain/islas-baleares/menorca/2020-06-30/data/calendar.csv.gz
    Path("calendar.csv.gz").rename("datasets/calendar.csv.gz")    
if not os.path.isfile('datasets/listings.csv'):
    !curl -OL http://data.insideairbnb.com/spain/islas-baleares/menorca/2020-06-30/visualisations/listings.csv
    Path("listings.csv").rename("datasets/listings.csv")    
if not os.path.isfile('datasets/reviews.csv'):
    !curl -OL http://data.insideairbnb.com/spain/islas-baleares/menorca/2020-06-30/visualisations/reviews.csv
    Path("reviews.csv").rename("datasets/reviews.csv")  
if not os.path.isfile('datasets/reviews.csv.gz'):
    !curl -OL http://data.insideairbnb.com/spain/islas-baleares/menorca/2020-06-30/data/reviews.csv.gz
    Path("reviews.csv").rename("datasets/reviews.csv.gz")     

## Как анализировать?

В календаре: номер объекта, дотсупность (false, true), цена, какая-то скорректированная цена, ограничение на количество ночей.

Что было бы интересно узнать?
Найти выгодные предложения, построить модель ценообразования, вообще получить представление о том, что там просходит.
Что можно посмотреть из календаря?
 - занятость по месяцам, отдельно по выходным
 - занятость относительно других объектов (тут уже надо подключать листинг и геолокацией)
 


In [50]:
cal_df = pd.read_csv("datasets/calendar.csv.gz",compression='gzip')

In [51]:
cal_df['available'] = cal_df.available.map(lambda x: 1 if x == 't' else 0)
cal_df.date = pd.to_datetime(cal_df.date)
cal_df['price'] = cal_df['price'].str.replace('$', '').str.replace(',', '')
cal_df['price'] = cal_df['price'].astype(float)
cal_df

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,44085,2020-06-30,0,350.0,$350.00,5.0,550.0
1,44085,2020-07-01,0,390.0,$390.00,5.0,550.0
2,44085,2020-07-02,0,390.0,$390.00,5.0,550.0
3,44085,2020-07-03,1,390.0,$390.00,5.0,550.0
4,44085,2020-07-04,1,390.0,$390.00,5.0,550.0
...,...,...,...,...,...,...,...
1183076,43630879,2021-06-25,0,185.0,$185.00,1.0,1125.0
1183077,43630879,2021-06-26,0,185.0,$185.00,1.0,1125.0
1183078,43630879,2021-06-27,0,185.0,$185.00,1.0,1125.0
1183079,43630879,2021-06-28,1,76.0,$76.00,1.0,1125.0


In [55]:
cal_df.info()
obj_44085 = cal_df[cal_df.listing_id==44085]
obj_44085

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183081 entries, 0 to 1183080
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   listing_id      1183081 non-null  int64         
 1   date            1183081 non-null  datetime64[ns]
 2   available       1183081 non-null  int64         
 3   price           1182911 non-null  float64       
 4   adjusted_price  1182911 non-null  object        
 5   minimum_nights  1182963 non-null  float64       
 6   maximum_nights  1182963 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 63.2+ MB


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,44085,2020-06-30,0,350.0,$350.00,5.0,550.0
1,44085,2020-07-01,0,390.0,$390.00,5.0,550.0
2,44085,2020-07-02,0,390.0,$390.00,5.0,550.0
3,44085,2020-07-03,1,390.0,$390.00,5.0,550.0
4,44085,2020-07-04,1,390.0,$390.00,5.0,550.0
...,...,...,...,...,...,...,...
360,44085,2021-06-25,1,600.0,$600.00,5.0,550.0
361,44085,2021-06-26,1,600.0,$600.00,5.0,550.0
362,44085,2021-06-27,1,600.0,$600.00,5.0,550.0
363,44085,2021-06-28,1,600.0,$600.00,5.0,550.0


In [66]:
obj_44085.groupby([obj_44085.date.dt.month])['available','price'].mean()

,available,price
date,,
1,0.000000,99.000000
2,0.000000,99.000000
3,0.000000,137.806452
4,0.000000,500.000000
5,1.000000,500.000000
6,0.966667,574.966667
7,0.935484,510.000000
8,1.000000,674.193548
9,1.000000,430.000000


 - The units digit (one decimal degree) gives a position up to 111 kilometers (60 nautical miles, about 69 miles). It can tell us roughly what large state or country we are in.
 - The first decimal place is worth up to 11.1 km: it can distinguish the position of one large city from a neighboring large city.
 - The second decimal place is worth up to 1.1 km: it can separate one village from the next.
 - The third decimal place is worth up to 110 m: it can identify a large agricultural field or institutional campus.
 - The fourth decimal place is worth up to 11 m: it can identify a parcel of land. It is comparable to the typical accuracy of an uncorrected GPS unit with no interference.
 - The fifth decimal place is worth up to 1.1 m: it distinguish trees from each other. Accuracy to this level with commercial GPS units can only be achieved with differential correction.

In [19]:
pd.read_csv("datasets/listings.csv")

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,44085,Villa in Addaia great sea view and fun pool,193043,Manuela,NaN,Es Mercadal,40.009460,4.200260,Entire home/apt,99,5,14,2018-08-25,0.14,4,166
1,102558,appartment in Menorca -Cala Galdana,536023,Yolanda,NaN,Ciutadella de Menorca,39.942730,3.962780,Entire home/apt,50,7,29,2019-08-03,0.26,1,88
2,148138,Apartment to rent in Ciutadella,715601,Jose Luis,NaN,Ciutadella de Menorca,40.002220,3.799190,Entire home/apt,49,1,18,2020-01-03,0.21,3,339
3,245722,Son Magna. Private intimacy in a farm in Alaior,1289210,Miguel,NaN,Alaior,39.896770,4.149120,Private room,200,3,19,2019-07-20,0.26,1,326
4,285865,Apartment sea view + swimming pool,1487752,Roberto,NaN,Ciutadella de Menorca,40.000880,3.799550,Entire home/apt,69,4,68,2019-10-18,0.70,1,307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236,43995938,ESPECTACULAR CASA EN CENTRO HISTÓRICO,6258490,Puri,NaN,Ciutadella de Menorca,40.003900,3.837180,Entire home/apt,100,6,0,NaN,NaN,12,356
3237,43997538,"Apartamento con piscina y vistas mar, Sol del ...",352526491,Jose Luis,NaN,Es Castell,39.874420,4.298490,Entire home/apt,120,6,0,NaN,NaN,1,135
3238,43999136,VILLA REGINA - CALAN BOSCH,6258490,Puri,NaN,Ciutadella de Menorca,39.925480,3.827970,Entire home/apt,240,7,0,NaN,NaN,12,73
3239,44004997,"Villacarlos, Hermoso lugar. Para conocer Menorca",276345511,Juan,NaN,Es Castell,39.879366,4.289337,Private room,45,1,0,NaN,NaN,1,348


In [71]:
reviews = pd.read_csv("datasets/reviews.csv.gz",compression='gzip')
reviews[reviews.listing_id == 44085]

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,44085,1639831,2012-07-06,1148512,Beatriz,Casa Binimasr es perfecta para unas vacaciones...
1,44085,4750726,2013-05-23,4180108,Fabrizio,The house was simply stunning! In a very quiet...
2,44085,5739873,2013-07-14,7168741,Cathelijne,We had a really nice stay at Binimasr. This wo...
3,44085,34596379,2015-06-09,6636779,Ana,La estancia en casa de Manuela fue muy agradab...
4,44085,46858585,2015-09-14,26646324,Monika,Manuela was extremely helpful even before we a...
5,44085,87145621,2016-07-18,62354198,Cécile,"Entrer chez Manuela, c'est comme rentrer chez ..."
6,44085,165914130,2017-07-02,6299439,Esther,We spent a blissful 5 days at Manuela's house ...
7,44085,169105277,2017-07-11,23731090,Paul,"We had a fantastic time at Manuela's villa, th..."
8,44085,171926856,2017-07-19,14811721,Javier,Una de las mejores cosas que se puede decir de...
9,44085,175666680,2017-07-29,15329113,Anita,Lovely villa. Small but perfectly clean and gr...


In [67]:
listing = pd.read_csv("datasets/reviews.csv")

In [70]:
listing[listing.listing_id == 44085]

,listing_id,date
0,44085,2012-07-06
1,44085,2013-05-23
2,44085,2013-07-14
3,44085,2015-06-09
4,44085,2015-09-14
5,44085,2016-07-18
6,44085,2017-07-02
7,44085,2017-07-11
8,44085,2017-07-19
9,44085,2017-07-29
